<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)  # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential()  # identity인 경우
        if stride != 1:  # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)  # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes  # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset

data_transformations = {
    "basic": transforms.Compose([
        transforms.ToTensor()
    ]),
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["crop_flip"])
augmented_train_data = ConcatDataset((train_dataset, train_aug_dataset))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 90:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.1640625
Current benign train loss: 2.305274248123169

Current batch: 100
Current benign train accuracy: 0.3203125
Current benign train loss: 1.7957367897033691

Current batch: 200
Current benign train accuracy: 0.34375
Current benign train loss: 1.7472918033599854

Current batch: 300
Current benign train accuracy: 0.3671875
Current benign train loss: 1.523087501525879

Current batch: 400
Current benign train accuracy: 0.5
Current benign train loss: 1.433241844177246

Current batch: 500
Current benign train accuracy: 0.4296875
Current benign train loss: 1.4712090492248535

Current batch: 600
Current benign train accuracy: 0.546875
Current benign train loss: 1.324449062347412

Current batch: 700
Current benign train accuracy: 0.390625
Current benign train loss: 1.547263503074646

Total benign train accuarcy: 40.941
Total benign train loss: 1271.0865201950073

[ Test epoch: 0 ]

Test accuarcy: 36.69
Test average loss: 


Test accuarcy: 85.72
Test average loss: 0.0042911916822195055
Model Saved!

[ Train epoch: 8 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.24051925539970398

Current batch: 100
Current benign train accuracy: 0.890625
Current benign train loss: 0.28179931640625

Current batch: 200
Current benign train accuracy: 0.859375
Current benign train loss: 0.38537904620170593

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.29942649602890015

Current batch: 400
Current benign train accuracy: 0.875
Current benign train loss: 0.40372949838638306

Current batch: 500
Current benign train accuracy: 0.8359375
Current benign train loss: 0.48206913471221924

Current batch: 600
Current benign train accuracy: 0.859375
Current benign train loss: 0.3670951724052429

Current batch: 700
Current benign train accuracy: 0.8828125
Current benign train loss: 0.2642296254634857

Total benign train accuarcy: 90.675
Total benign train


Total benign train accuarcy: 93.359
Total benign train loss: 151.9478858076036

[ Test epoch: 15 ]

Test accuarcy: 87.54
Test average loss: 0.004015291787683964
Model Saved!

[ Train epoch: 16 ]

Current batch: 0
Current benign train accuracy: 0.9609375
Current benign train loss: 0.13028597831726074

Current batch: 100
Current benign train accuracy: 0.9296875
Current benign train loss: 0.23576410114765167

Current batch: 200
Current benign train accuracy: 0.953125
Current benign train loss: 0.1829240471124649

Current batch: 300
Current benign train accuracy: 0.890625
Current benign train loss: 0.29711177945137024

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.16476474702358246

Current batch: 500
Current benign train accuracy: 0.890625
Current benign train loss: 0.26904040575027466

Current batch: 600
Current benign train accuracy: 0.9140625
Current benign train loss: 0.23718014359474182

Current batch: 700
Current benign train accuracy: 0.91


Current batch: 700
Current benign train accuracy: 0.953125
Current benign train loss: 0.10819695144891739

Total benign train accuarcy: 94.155
Total benign train loss: 132.24569971114397

[ Test epoch: 23 ]

Test accuarcy: 87.23
Test average loss: 0.004175129541754722
Model Saved!

[ Train epoch: 24 ]

Current batch: 0
Current benign train accuracy: 0.953125
Current benign train loss: 0.18565607070922852

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.11665430665016174

Current batch: 200
Current benign train accuracy: 0.9375
Current benign train loss: 0.15418055653572083

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.21578988432884216

Current batch: 400
Current benign train accuracy: 0.96875
Current benign train loss: 0.1417401134967804

Current batch: 500
Current benign train accuracy: 0.9375
Current benign train loss: 0.15707121789455414

Current batch: 600
Current benign train accuracy: 0.984375
Cur


Current batch: 600
Current benign train accuracy: 0.9375
Current benign train loss: 0.18162226676940918

Current batch: 700
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09939856082201004

Total benign train accuarcy: 94.517
Total benign train loss: 123.91242941841483

[ Test epoch: 31 ]

Test accuarcy: 87.64
Test average loss: 0.004030177317559719
Model Saved!

[ Train epoch: 32 ]

Current batch: 0
Current benign train accuracy: 0.96875
Current benign train loss: 0.1182660460472107

Current batch: 100
Current benign train accuracy: 0.953125
Current benign train loss: 0.15980345010757446

Current batch: 200
Current benign train accuracy: 0.9296875
Current benign train loss: 0.18808381259441376

Current batch: 300
Current benign train accuracy: 0.9609375
Current benign train loss: 0.10856768488883972

Current batch: 400
Current benign train accuracy: 0.9609375
Current benign train loss: 0.09410173445940018

Current batch: 500
Current benign train accuracy: 0.976

...


Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0045331260189414024

Current batch: 300
Current benign train accuracy: 0.984375
Current benign train loss: 0.04751400277018547

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.003183274529874325

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0025842462200671434

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0019296108512207866

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.006460925564169884

Total benign train accuarcy: 99.806
Total benign train loss: 5.95291410892969

[ Test epoch: 63 ]

Test accuarcy: 94.39
Test average loss: 0.0022143426584079863
Model Saved!

[ Train epoch: 64 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0016472392017021775

Current batch: 100
Current benign train accuracy: 1.0
Current benign train l


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0014623564202338457

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0030742217786610126

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0013797036372125149

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0017293622950091958

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.00209637638181448

Total benign train accuarcy: 99.874
Total benign train loss: 4.022507777088322

[ Test epoch: 71 ]

Test accuarcy: 94.57
Test average loss: 0.002095156516879797
Model Saved!

[ Train epoch: 72 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0006468280917033553

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0017865928821265697

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0015370685141533613

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.008133702911436558

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.018350964412093163

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0038644412998110056

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0023990522604435682

Total benign train accuarcy: 99.895
Total benign train loss: 3.399857204844011

[ Test epoch: 79 ]

Test accuarcy: 94.38
Test average loss: 0.0022153208017349243
Model Saved!

[ Train epoch: 80 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.002780989045277238

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0007936640176922083

Current batch: 200
Current benign train accuracy: 1.0
Current benign trai


Current batch: 300
Current benign train accuracy: 0.9921875
Current benign train loss: 0.01636875607073307

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.00861818715929985

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.006450968328863382

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0013508235570043325

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0057544950395822525

Total benign train accuarcy: 99.821
Total benign train loss: 5.304003526864108

[ Test epoch: 87 ]

Test accuarcy: 94.32
Test average loss: 0.0023529045306146146
Model Saved!

[ Train epoch: 88 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0039931293576955795

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0014831004664301872

Current batch: 200
Current benign train accuracy: 0.9921875
Current b


Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0012016886612400413

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.001984004396945238

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0005168740171939135

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.005974677391350269

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0039023144636303186

Total benign train accuarcy: 99.969
Total benign train loss: 1.5835278537124395

[ Test epoch: 95 ]

Test accuarcy: 94.7
Test average loss: 0.0021702968530356885
Model Saved!

[ Train epoch: 96 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0045514460653066635

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0008948463364504278

Current batch: 200
Current benign train accuracy: 1.0
Current benign train los